In [1]:
import pandas as pd
import database_utils as dbu

![](database_tables.png)

## 1. Find the day when the most chicken was sold at location 3

In [5]:
database = 'supermarket.db'

query = '''
           select date(timestamp), product, location, sum(quantity)
           from transactions

           where product = 'chicken'
           and location = 'location_3'

           group by 1, 2, 3
           order by 4 desc
           limit 1
        '''
dbu.query_sql(database=database, sql=query)

Query executed successfully!
query returned 1 rows


,0,1,2,3
0,2021-03-02,chicken,location_3,79.0


## 2. Find the location that sells the most meat products.

In [7]:
database = 'supermarket.db'

query = '''
           select location, category, sum(quantity)
           from transactions t1

           left join products t2 on t2.product = t1.product

           where category = 'meat'

           group by 1, 2
           order by 3 desc
        '''
dbu.query_sql(database=database, sql=query)

Query executed successfully!
query returned 5 rows


,0,1,2
0,location_2,meat,6741.0
1,location_1,meat,4686.0
2,location_5,meat,4444.0
3,location_4,meat,4288.0
4,location_3,meat,3845.0


## 3. What is the average number of meat products sold by the supermarket.

- Note that this question is deliberately ambiguous. The average can be calcualted based different groupings e.g daily average, weekly average, average by location etc. Bonus points for a student to recognize this. However, one can also simply calculate the overall average across all transactions as shown in query1. query2 on the other hand shows how you one can calculate the daily average.

In [13]:
database = 'supermarket.db'

query1 = '''
           select category, avg(quantity)
           from transactions t1

           left join products t2 on t2.product = t1.product

           where category = 'meat'

           group by 1
        '''

query2 = '''
           with daily_meat as (
                select date(timestamp), category, sum(quantity) as total_meat
                from transactions t1

                left join products t2 on t2.product = t1.product

                where category = 'meat'

                group by 1, 2
           )

           select avg(total_meat)
           from daily_meat
        '''

dbu.query_sql(database=database, sql=query2)

Query executed successfully!
query returned 1 rows


,0
0,774.322581


## 4. List all the days when the number of meat products sold at location 5 was below average.

In [18]:
database = 'supermarket.db'

query = '''
           with daily_meat as (
                select date(timestamp) as day, category, sum(quantity) as total_meat
                from transactions t1

                left join products t2 on t2.product = t1.product

                where category = 'meat'

                group by 1, 2
           ),

           average_meat as (
                select avg(total_meat) as average_value
                from daily_meat
           )

           select day, total_meat
           from daily_meat
           where total_meat < (select average_value from average_meat)
        '''

dbu.query_sql(database=database, sql=query)

Query executed successfully!
query returned 17 rows


,0,1
0,2021-03-03,684.0
1,2021-03-04,683.0
2,2021-03-06,659.0
3,2021-03-08,729.0
4,2021-03-09,729.0
5,2021-03-11,705.0
6,2021-03-13,657.0
7,2021-03-15,737.0
8,2021-03-17,684.0
9,2021-03-18,706.0


## 5. For each location, what is the busiest hour of the day?
- A reasonable measure for how busy a location is can be the total number of transactions. However, depending on the specific business context considered, a different measure may be preferred.

In [32]:
database = 'supermarket.db'

query = '''
        select hour_of_day, location, max(num_transactions)

        from (
                select time(timestamp) as hour_of_day, location, count(1) as num_transactions
                from transactions
                group by 1, 2
              )
        
        group by location
        '''
dbu.query_sql(database=database, sql=query)

Query executed successfully!
query returned 5 rows


,0,1,2
0,18:00:00,location_1,246
1,18:00:00,location_2,371
2,18:00:00,location_3,258
3,17:00:00,location_4,272
4,18:00:00,location_5,208


## 6. Which location has the busiest weekends?
Again, we can consider the total number of transactions to be a good indicator of how bust a location was. As a result, if we calculate the total number of transactions at each location on weekends, we can compare and make a judgement about the busiest location.

In [37]:
database = 'supermarket.db'

query = '''
        select location, count(1)
        from transactions

        where strftime('%w', timestamp) in ('0','6') -- sunday is 0 and saturday is 6
        
        group by 1
        order by 2 desc
        '''
dbu.query_sql(database=database, sql=query)

Query executed successfully!
query returned 5 rows


,0,1
0,location_2,773
1,location_4,494
2,location_3,470
3,location_1,464
4,location_5,426
